In [20]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
import pandas_ta as ta
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
%matplotlib inline
from decimal import *
getcontext().prec = 6
import seaborn as sns

matplotlib.rcParams.update({'font.size': 5, 'lines.linewidth': 0.5, 'figure.dpi': 150})


In [10]:
def load_candles(pair='BTC_USDT', timeframe='1d'):
    odf = pd.read_json(f'../../freq-user-data/data/binance/{pair}-{timeframe}.json').dropna()
    odf.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

    odf['date'] = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)
    # odf['idate'] = odf.date.dt.strftime('%Y%m%d')
    odf.set_index(pd.DatetimeIndex(odf["date"]), inplace=True, drop=True)
    # odf = odf.sort_index()
    odf = odf[['open', 'high', 'low', 'close']]
    return odf

In [11]:
odf = load_candles(pair='BTC_USDT', timeframe='4h')
odf.head()


,open,high,low,close
date,,,,
2017-08-17 04:00:00,4261.48,4349.99,4261.32,4349.99
2017-08-17 08:00:00,4333.32,4485.39,4333.32,4427.30
2017-08-17 12:00:00,4436.06,4485.39,4333.42,4352.34
2017-08-17 16:00:00,4352.33,4354.84,4200.74,4325.23
2017-08-17 20:00:00,4307.56,4369.69,4258.56,4285.08


In [25]:

def plot_candles(wdf, ax=None, kwargs={}):
    if ax is None:
        fig, ax = plt.subplots(**kwargs)

    up, down = wdf[wdf.close >= wdf.open], wdf[wdf.close < wdf.open]
    col1,col2 = 'green','red'
    width, width2 = .1, .02
    # Plotting up prices of the stock
    ax.bar(up.index, up.close-up.open, width, bottom=up.open, color=col1)
    ax.bar(up.index, up.high-up.close, width2, bottom=up.close, color=col1)
    ax.bar(up.index, up.low-up.open, width2, bottom=up.open, color=col1)
    # Plotting down prices of the stock
    ax.bar(down.index, down.close-down.open, width, bottom=down.open, color=col2)
    ax.bar(down.index, down.high-down.open, width2, bottom=down.open, color=col2)
    ax.bar(down.index, down.low-down.close, width2, bottom=down.close, color=col2)    
    plt.xticks(rotation=30, ha='right')
    return ax

In [33]:

from matplotlib import pyplot as plt

strategy = 'persistence'
param_folder = f'.'
strategy_params_json = f'{param_folder}/{strategy}.json'

dlen = len(odf)


sl_n2= IntSlider(description="n2", min=0, max=ceil(log(dlen, 2.0)+1), step=1, value=7)
sl_w = IntSlider(description="w", min=0, max=ceil(dlen/pow(2,sl_n2.value-1)-1), step=1, value=1)
def update_sl_w_range(*args):
    sl_w.max = ceil(dlen/pow(2,sl_n2.value-1)-1)
sl_n2.observe(update_sl_w_range, 'value')

sl_hold = IntSlider(value=7,min=1,max=50,step=1,description='Hold',continuous_update=False)
sl_lag_n = IntSlider(value=5,min=1,max=50,step=1,description='lag_n',continuous_update=False)
# tx_max_rat = FloatText(value=0.05,min=0.01,max=0.1,step=0.001,description='max_rat',continuous_update=False)
sl_lag = IntSlider(value=6,min=1,max=6 * 7,step=1,description='Lag',continuous_update=False)
chk_candles = Checkbox(value=True, description='Candles', disabled=False)
dd_dir = Dropdown(options=[('Up', 1), ('Down', -1)], value=1, description='Dir',)

strategy_params = {
    "candles": {"wdg": chk_candles},
    "lag": {"wdg": sl_lag},
}
all_params = {
    'w2log': {'wdg': sl_n2}, 'w': {'wdg': sl_w},
    **strategy_params
}
wdgts = [pv['wdg'] for pk, pv in strategy_params.items()]

ui = widgets.VBox([
    widgets.HBox([sl_n2, sl_w]),
    widgets.VBox([widgets.HBox(wdgts[i:i+4]) for i in range(0, len(wdgts), 4)])
])

if os.path.exists(strategy_params_json):
    with open(strategy_params_json) as f: 
        js = json.loads(f.read());
        for k, v in all_params.items(): 
            if k in js: v['wdg'].value=js[k];

else: print(f'File not found: {strategy_params_json}')

wdf = None

def printer(
        w2log, w, candles, lag
):
    
    global wdf
    
    with open(strategy_params_json, "w") as f: f.write(json.dumps({k: v['wdg'].value for k, v in strategy_params.items()}))
    
    
    wsz = floor(pow(2,w2log))
    wst = floor(w * wsz / 2)
    wed = floor(wst + wsz)
    
    wdf = odf.iloc[wst:wed,:].copy()    
    wlen = len(wdf)

    print(f'N={len(wdf)}; Period: {wdf.index[-1] - wdf.index[0]}, Start: {wdf.index[0]}, End: {wdf.index[-1]}\n')
    # print(f'lag_n={lag_n}; min_rat={min_rat}; max_rat={max_rat}\n')


    fig = plt.figure(layout='constrained', figsize=(15,5))
    fig1,fig2 = fig.subfigures(2,1, wspace=0.001, hspace=0, height_ratios=[3,1])

# axsLeft = subfigs[0].subplots(1, 2, sharey=True)

    (ax1) = fig1.subplots(1, 1, height_ratios=[1], sharex=True);

    if not candles:
        wdf.close.plot(ax=ax1)
    else:
        plot_candles(wdf=wdf,ax=ax1)
    ax1.plot()

out = widgets.interactive_output(printer, {k : v['wdg'] for k,v in all_params.items()});
x = display(ui, out);






Output()